In [ ]:
import pandas as pd
import numpy as np 
import sys 
import warnings
import tensorflow as tf
from pathlib import Path
import keras
from keras import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D, Input
from keras.layers import Convolution2D, MaxPooling2D, BatchNormalization, GlobalMaxPooling2D, Concatenate
import os
from google.colab import drive
drive.mount('/content/drive')
sys.path.append('..')
warnings.filterwarnings("ignore")
np.random.seed(2020)

In [ ]:
#load tensorboard
%load_ext tensorboard
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
#see if we are in the right dir
dir_root = "/"
os.listdir(dir_root)

In [ ]:
#see if we are in the right dir
os.listdir(dir_root+ "train")

In [ ]:
#calculate the weight of three classes by their frequencies
def weight(data_root_dir):
    leaf_rust_count = len(list(Path(data_root_dir + "train/leaf_rust").glob('*')))
    stem_rust_count = len(list(Path(data_root_dir + "train/stem_rust").glob('*')))
    healthy_wheat_count = len(list(Path(data_root_dir + "train/healthy_wheat").glob('*')))
    total = leaf_rust_count + stem_rust_count + healthy_wheat_count

    healthy_wheat_weight = (1/healthy_wheat_count) * (total) / 3.0
    leaf_rust_weight = (1/leaf_rust_count) * (total) / 3.0
    stem_rust_weight = (1/stem_rust_count) * (total) / 3.0
    
    class_weight = {0:healthy_wheat_weight, 1:leaf_rust_weight, 2:stem_rust_weight}
    return class_weight
    
class_weight=weight(dir_root)

In [ ]:
#construct our transfer learning model from DenseNet-201 in Keras
from keras.applications.densenet import DenseNet201
def densenet201():
    base_model = DenseNet201(include_top=False, weights='imagenet')    #do not need the top layer of DenseNet-201
    x = base_model.output              #get the output tensor of DenseNet-201
    x=GlobalMaxPooling2D()(x)      #pooling
    x=BatchNormalization()(x)         #relieve overfitting
    x=Dense(256, activation='relu')(x)     #fc layer to learn more information about the activation map of the transfer learning model
    x=BatchNormalization()(x)
    x=Dense(128, activation='relu')(x)
    x=BatchNormalization()(x)
    preds=Dense(3,activation='sigmoid')(x) #final layer with sigmoid activation
    model = Model(inputs=base_model.input, outputs=preds)  #the input is that of the base_model and the output shape will be our prediction shape
    return model

In [ ]:
#the data generator to generate augmented image data in order to relieve overfitting
train_datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    rotation_range=30,  # randomly rotate images in the range (degrees, 0 to 180)
    # randomly shift images horizontally (fraction of total width)
    width_shift_range=0.15,
    # randomly shift images vertically (fraction of total height)
    height_shift_range=0.15,
    shear_range=0.1,  # set range for random shear
    zoom_range=0.15,  # set range for random zoom
    channel_shift_range=0.0,  # set range for random channel shifts
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    cval=0.,  # value used for fill_mode = "constant"
    horizontal_flip=True,  # randomly flip images
    vertical_flip=True,  # randomly flip images
    # set rescaling factor (applied before any other transformation)
    rescale=1./255,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format='channels_last'
)

#for validation data, we just need to rescale without data augmentation
validation_datagen = ImageDataGenerator(
    rescale= 1./255,
    data_format='channels_last'
)

In [ ]:
#early stopping callback
callback = [keras.callbacks.EarlyStopping(monitor='val_loss',patience=4)]

batch_size = 8
#the image data and its label will flow from its directory, with a predefined batch size
#shullfe=True is by default
train_generator = train_datagen.flow_from_directory(
    dir_root + 'train',
    target_size=(224, 224),
    batch_size= batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    dir_root + 'validation',
    target_size=(224, 224),
    batch_size= batch_size,
    class_mode='categorical' 
)   

In [ ]:
 model = densenet201()
model.compile(loss='categorical_crossentropy',
                optimizer=keras.optimizers.adam(lr=1e-4),  
                metrics=['accuracy'])

#fitting generator will save the memory consumed by our program. we include class_weight here to address the imbalanced dataset problem
model.fit_generator(
            train_generator, 
            steps_per_epoch = 700//batch_size+1,
            epochs=20 ,
            validation_data = validation_generator,
            validation_steps = 176//batch_size+1,
            callbacks=callback,
            class_weight=class_weight)
 
 learning_rates = [1e-4, 1e-5, 1e-6]
 for lr in learning_rates: 
    #after having the data and architecture model, we need to compile it with loss function, optimizer and metrics
    model.compile(loss='categorical_crossentropy',
                    optimizer=keras.optimizers.SGD(lr=lr, momentum=0.9),  
                    metrics=['accuracy'])

    #fitting generator will save the memory consumed by our program. we include class_weight here to address the imbalanced dataset problem
    model.fit_generator(
                train_generator, 
                steps_per_epoch = 700//batch_size+1,
                epochs=20 ,
                validation_data = validation_generator,
                validation_steps = 176//batch_size+1,
                callbacks=callback,
                class_weight=class_weight)

In [ ]:
#creating the submission csv file
def create_submission(model, name):
    #import the Image module
    from PIL import Image
    def load_process(path):
        path_str = str(path)
        image = Image.open(path_str)
        image.load()
        image = np.asarray(image, dtype="float32" ) #change the image into the nd-array
        image /= 255.0 #rescale the image by dividing 255
        image = image.reshape(-1,224,224,3) #add a more dimension
        name = path.name.split('.')[0] #pick the name
        return name, image

    paired_name_image = list(map(load_process, list(Path(dir_root +'/test').glob('*'))))
    index = [i[0] for i in paired_name_image]
    probs = [model.predict(i[1])[0] for i in paired_name_image]
    submission = pd.DataFrame(probs)
    submission.index = index
    submission.columns = ['healthy_wheat', 'leaf_rust', 'stem_rust']
    submission = submission[['leaf_rust', 'stem_rust', 'healthy_wheat']]
    submission.columns = ['leaf_rust', 'stem_rust', 'healthy_wheat']   #change the column order
    submission.to_csv("/content/submission_"+name+".csv")  #save the csv file into /content dir
create_submission(model, "DenseNet201")

In [ ]:
# load json and create model
def load_model(name):
    from keras.models import model_from_json
    json_file = open("/"+name+".json", 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("/"+model+".h5")
    print("Loaded model from disk")
 
# save the model into json file and serialize the weights into h5 file
def save_model(model, name):
    """
    the first arg is the model object and the second arg is the name that you would like to store
    """
    model_json = model.to_json()
    with open("/"+name+".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("/"+name+".h5")
    print("Saved model to disk")
 
#save_model(model, "densenet201")